In [1]:
# from torch import nn
import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers.optimization import AdamW
# from tqdm import trange, tqdm_notebook

In [2]:
from transformers.modeling_camembert import CamembertForQuestionAnswering
from transformers.tokenization_camembert import CamembertTokenizer

C:\Program Files\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Program Files\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Program Files\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Program Files\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [3]:
camembert = CamembertForQuestionAnswering.from_pretrained('fmikaelian/camembert-base-fquad')
tokenizer = CamembertTokenizer.from_pretrained('fmikaelian/camembert-base-fquad'
                                               , do_lower_case=True
                                              )

In [4]:
def split_text(context, length = 512,overlapp= 0):     # onlyText
    res = []
    tokens = tokenizer.encode(context, add_special_tokens=True)
    for i in range(0, int(len(tokens)-overlapp),(length-overlapp)):
            text = tokenizer.decode(tokens[i:i+length])
            text = text.replace('<s>','').replace('</s>','')
#             text = text.replace('</s>','')
#             print(text)
            res.append(text)
    return res
# print(split_text(text, overlapp=10))


# def split_text(question, context, overlapp= 0):      # Question and text
#     res = []
#     tokens = tokenizer.encode(question, context, add_special_tokens=True)
#     for i in range(0, int(len(tokens)-overlapp),(512-overlapp)):
#             question, text = tokenizer.decode(tokens[i:i+512])
# #             text = text.replace('<s>','')
# #             text = text.replace('</s>','')
            
#             print(text)
#             print("-------------------------")
# #             res.append(tokens[i:i+512])
#     return res



In [55]:
# For each paragraph answer the question
def select_answer(question, text):
    # encode the question
    encoded_question = tokenizer.encode(question,add_special_tokens=True)
    length = len(encoded_question)
    encoded_text = split_text(context = text, length = 509-length)   # add overlapping    510
    max_score = 0
    answer = ""
    for split in encoded_text:
#         print(split)
        inp = tokenizer.encode(question, split, add_special_tokens=True)
        input_ids = torch.tensor([inp])
        with torch.no_grad():
            last_hidden_states = camembert(input_ids)[0]  # Models outputs are now tuples
            score = torch.max(last_hidden_states)
            if (max_score < score):
                # Update the answer
                max_score = score
                all_tokens = tokenizer.convert_ids_to_tokens(inp)
                start_scores, end_scores = camembert(input_ids)
                start = torch.argmax(start_scores)
                end = torch.argmax(end_scores)+1
                answer = "".join(all_tokens[start:end]).replace('▁',' ').strip()
#             print(answer)
    return answer

# Corona wikipédia 

# RQPS

In [50]:
# Preprocessing text
import re
def prepro(text):
    text = re.sub("[|*°«»¢]","",text).lower()
    text = re.sub(" oo ", " ",text)
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)
    text = re.sub(r' ,', r',', text)
    text = re.sub('//', '', text)
    text = re.sub('\s*([\'’])\s*', r'\1', text)
    text = re.sub('\s*([.,!?$£€])\s*', r'\1 ', text)
    text = re.sub(r' %', r'%', text)
    #remove websites
#     text = re.sub(r' ’ ', r'\'', text)
    return text

# print(prepro(text))

In [57]:
text = """
EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE VOTRE COMMISSION LOCALE VOTRE PRESIDENT Francis FRANCK Au cours de l'année 2018 , la dynamique d'adhesions et d'intéegrations au SDEA s'est poursuivie , traduisant ainsi la confiance croissante exprimée a l'egard de notre outil commun par de nombreuses communes et établissements publics de coopération intercommunale EPC . Le SDEA compte ainsi désormais 668 communes membres au total , fedérant plus de 1 000 000 d'habitants . Ces adhesions , dans les domaines historiques de I'Eau Potable et |'Assainissement , mais aussi dans le domaine plus recent du Grand Cycle de I'Eau , permettent a la fois une capacité renforcee de mutualisation au SDEA au benéfice de tous nos usagers , et une opportunite de repondre aux enjeux de l'eau dans les différents territoires , en partenariat avec les intercommunalités et les acteurs cles . Dans le contexte marque par le changement climatique et les nouveaux défis environnementaux nécessitant une approche encore plus intégree du cycle de l'eau , il s'agit egalement de conforter I'excellence technique et économique de l'ensemble des interventions du SDEA au service de l'interét general . D'importantes economies sont ainsi obtenues permettant de réinvestir d'autant plus dans le renouvellement et l'amélioration du patrimoine local . VOS USAGERS PERIMETRE oe 4936 abonnés 19575 habitants desservis Périmétre créé le 01/01/2018 Membre du SDEA depuis 01/01/2018 VOS VOLUMES 1161630 m consommés 59 m* consommeés/habitant 235 m* consommés/abonné Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 1

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE VOTRE PATRIMOINE puits dont 1 puits non autorisé ne disposant pas de DUP Aucune source » 1 réservoir capacité totale de stockage 500 m ’ Aucune station de traitement 1 unité de désinfection » 103,698 km de conduites CARACTERISTIQUES DES OUVRAGES Capacité de production Capacité journaliére maxi 18 000 m*/jour Volume prélevé journalier moyen 5 555 m°/jour Taux de mobilisation jour moyen 31 % Volume prélevé journalier de pointe 6 535 m*/jour » Taux de mobilisation jour de pointe 36 % Capacite de stockage Volume utile des réservoirs 500 m Autonomie réservoir en moyenne 0,1 jour s » Nombre d ’ heures d ’ autonomie réservoir en pointe 2 h Indicateurs de performance Indice de connaissance et de gestion patrimoniale des reseaux d ’ eau potable Indice d ’ avancement de la protection de la ressource en eau Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 2

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE LA QUALITE DE VOTRE EAU CHIFFRES CLES Durete de l ’ eau °F Env . 26 Nitrates mg/l Env . 11 CONTROLE REGLEMENTAIRE ARS 2017 2018 Nombre de prelevements analyses microbiologiques total Nombre de prélévements analyses microbiologiques non conformes po oo Nombre de prélevements analyses physico-chimiques total Pour plus d ’ informations sur la qualité de l'eau , vous pouvez consulter la Synthese annuelle de l'Agence Regionale de la Sante sur http /Awww.sdea.fr/index.php/Usagers/L-eau-dans-la-commune.html Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 3

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE VOS DONNEES FINANCIERES PRIX DE VOTRE EAU Retrouvez ci-dessous les elements constitutifs du prix de l'eau sur votre périmétre Prix de l'eau par m pour 120 m norme INSEE Part fixe 37,44 €HT/an Part variable 0,733 €HT le m Redevance eau potable du périmetre 1,045 €HT par m pour 120 m Vv Vv V WV Prix du service eau potable , redevances Agence de I|'Eau et TVA comprises 1,53 €TTC par m pour 120 m 9 872 factures emises dans l ’ année Prix de l'eau par m pour 120 m =m Redevance Eau Potable TVA 5,5 % =m Redevance prélevement Redevance pollution domestique Affectation pour 100 € de recette Dépenses d'exploitation Remboursement de la dette a Autofinancement Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 4

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE Montant des investissements 500 000 € 400 000 € 300 000 € 200 000 € 100 000 € O€ O€ 2016 2017 2018 416 966 € En € HT/an Evolution des tarifs de l'eau 1,20 € 1,05 € 1,05 € 1,00 € 0,80 € 0,60 € 0,40 € 0,20 € 0,00 € En € HT/m 2016 2017 2018 Indicateurs financiers Durée d ’ extinction de la dette de la collectivité ND Capital restant dd Taux d ’ impayés sur factures d ’ eau de l'année precédente Montant des abandons de creances Taux de reclamations global 0,15 % 0 Le périmetre de Sélestat eau potable degage un résultat d'exploitation excédentaire de 862k€ . Les déepenses d'exploitation sont maitrisees et en dessous du budget prévisionnel . Concernant les recettes d'exploitation , elles sont superieures a celles budgétées et reprennent l'intégration du resultat de transfert a hauteur de 275k€ . Les travaux d'investissement sont integralement autofinancés par le périmetre . Le resultat de cloture pour l'année 2018 est excédentaire de prés de 550k€ € et la durée d'extinction de la dette inférieure a 1 an . Pour plus d ’ informations sur les redevances , vous pouvez consulter la note d'information annuelle de Agence de I ’ Eau Rhin-Meuse sur http // www.eau-rhin-meuse . fr Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 5

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE VOTRE RESEAU D ’ EAU POTABLE PRODUCTION - VENTES 2 500 000 2 000 000 m™ Volume d'eau non facturé/pertes c m= Volume autorisé non compté - 1500000 oO g Volume exporté 1000000 m Volume d'eau facturé aux gros consommateurs 500 000 m Volume d'eau facturé aux abonnés hors gros consommateurs 0 2016 2017 2018 INTERRUPTIONS DE SERVICE ET RUPTURES Indicateurs de performance 2018 Delai maximal d'ouverture des branchements pour les nouveaux abonnés deéfini/service 48h Taux d'occurrence des interruptions de service non programmeées 2,84 % 0 Taux de respect du délai maximal d'ouverture des branchements pour les nouveaux abonnes 100 % Vina Indice linéaire de réparation nb/km Année Indice linéaire des volumes non comptes m /km/j Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 6

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE APPAREIL DE DEFENSE CONTRE L'INCENDIE poteaux et hydrants 765 appareils 3réparations sur les poteaux d ’ incendie 1remplacement PARC DE COMPTEURS 5012 nombre total de compteurs 35 compteurs remplacés , soit 0,7 % du parc 97 % de compteurs radio-relevés NOMBRE DE RUPTURES REPAREES 45 41 40 35 30 m Nombre de ruptures 20 18 branchements 20 15 14 m Nombre de ruptures conduites principales 10 6 ee 0 2016 2017 2018 RENDEMENT ET ILP DES RESEAUX 80 72,7 72 72 20 « o oN 60 15 3 . 8 oO 4 ~~ oO Cc , c OM . © AO 10 5 2 5 35 0 3,0 SG 20 5 se O 33 —S 0 5 FF 2016 2017 2018 x Gam Rendement en % em dice linéaire de pertes en réseau Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 7

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE LES INVESTISSEMENTS SUR VOS RESEAUX ET OUVRAGES INTERVENTIONS PRINCIPALES SUR RESEAUX ET OUVRAGES Les travaux de rénovation et renforcement hydraulique des reseaux du Perimetre de Sélestat se sont poursuivis en 2018 a SELESTAT Rue du Cimetiere Renforcement du debit decennal - 520 ml DN 150 - Renouvellement de 30 branchements particulier . suite a programme de voirie Chemin de Scherwiller Extension du réseau d'assainissement - 180 ml DN100 Rue des Glycines Renovation du réseau d'eau potable - 190 ml DN 100 - Renouvellement de 13 branchements particulier . suite a programme de voirie . Les travaux de renovation et renforcement hydraulique des reseaux du Périmetre de Sélestat seront encore prolonges dans les 5 années a venir dans le cadre d'une programmation pluriannuelle . 2019 Rue Johann Strauss Renovation du réseau d'eau potable - 180 ML DN100 - Reprise de 17 branchements particuliers gestion patrimonial Rue de la Forét Renovation du reseau d'eau potable - 300 ml DN Reprise de 30 branchements particulier Rue Antoine Jean Jean Renovation du réseau d'eau potable - 170 ml - Reprise des 10 branchements particuliers Rue Renouvier Gouraud Renforcement du reseau eau potable - 140ml en DN 250 et 130ml en DN 150 - Reprise de 20 branchements particuliers Rue du Giessen Renforcement du réseau eau potable - 80 ml en DN150 et 80 ml en DN 100 EVOLUTION DES TRAVAUX SUR RESEAUX Gl Rénovation/amélioration de conduites principales 800 690 1 ay 2 600 c c o 400 05 5B E of S 200 oS © 0 oO 0 oO 0 o © 0 0 0 ¢€°0 O Cc wv ov 2016 2017 2018 `` 3 3 2 oO © Ml Extensions des conduites principales ese Taux moyen sur 5 ans de renouvellement des réseaux Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 8

EAU POTABLE 2018 PERIMETRE DE SELESTAT TERRITOIRE ALSACE CENTRALE VOTRE ACTUALITE ZOOM SUR TRAVAUX EFFECTUES ET A VENIR Les principales operations programmees en 2019 concernent Travaux de renouvellement du réseau d'eau potable seuls gestion patrimoniale e rue Antoine Jean Jean , coit estimé 100 000 € HT e rue Johann Strauss , codt estimé 105 000 € HT e rue du Giessen partie , cout estime 95000 € HT Travaux de renouvellement du reseau d ’ eau potable en commun avec des travaux d ’ assainissement e rue Renouvier , rue Gouraud , cout estimé 150 000 € HT Travaux de renouvellement du réseau d'eau potable avant réfection lourde de voirie e rue dela Forét , coat estimé 200 000 € HT SELESTAT — Rue des Glycines Renforcement du réseau d ’ eau potable Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 9

EAU POTABLE 2018 a TERRITOIRE ALSACE CENTRALE SELESTAT — Rue du Cimetiére Renforcement du réseau d ’ eau potable Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 10

ere ale =11 83 A Gi PV . S 2a ns tc PORTEE DE N 2 veal Ss i cr ~— - oo 5 G10 GG a 2 ee S| LO 039 o U aja 402 6i- 21a . oropose UN acces aU 2 jee cal COMMENT TELECHARGER UAPPLICATION MOBILE SDEA Depuis une plateforme Android ou iOS , vous recherchez « SDEA » sur Play Store ou App Store et cliquez sur « Installer » . Le tour est joué , vous pouvez prévenir instantanément votre service public de l'eau en cas d'urgence . QUEL DYSFONCTIONNEMENT SIGNALER Une odeur ou un godt inhabituel de l'eau Une fuite d ’ eau ou un manque d ’ eau Une borne incendie endommagée ou une grille manquante Une inondation par refoulement des eaux usées Votre signalement bien été envoyeé Le SDEA en prendra connaissance dans les plus brefs délais Vous recevrez alors un accusé de réception par email En mages Prendre une DEVENEZ ACTEUR DU SERVICE DE UVEAU EN TOUTE SIMPLICITE , VOUS POUVEZ SIGNALER TROUVER vam lies mea 1 le SDEA pres dysfonctionnement de chez vous OBTENIR DES CONSEILS adaptés a vos besoins \ CONTACTER Se SUT ge TdT -e 24h/24 , 75/7 CONSULTER les vidéos du SDEA ACCEDER a historique de vos signalements a Oba bry “ ll applicatior SDEA ces une iTo ih aielae appli mobile évoluera pour vous proposer bientdt de nouvelles fonctionnalités liées notamment a la gestion de l ’ abonnement reléve de l ’ index , suivi conso , paiement de la facture ainsi qu ’ a la prévention des inondations et coulées boueuses . Le SDEA prés de chez moi Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - video Galerie vidéo Géret vos pléces jointes OK renvoi vers le glossaire- 11

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE GLOSSAIRE LISTE DES ABREVIATIONS ET DEFINITIONS EP Eau Potable ARS Agence Regionale de Sante AERM Agence de I ’ Eau Rhin-Meuse Pl Perimetre Intégre PP — Périmétre Partiellement Integre UD — Unite de distribution INDICATEURS DE PERFORMANCE - source http //www.services.eaufrance.fr/observatoire/indicateurs Indice de connaissance et de gestion patrimoniale des réseaux d ’ eau potable Cet indicateur évalue sur une echelle de 0 a 100 , a la fois le niveau de connaissance du réseau et des branchements et l ’ existence d ’ une politique de renouvellement pluri-annuelle du service d'eau potable - Formule de calcul Voir la fiche descriptive complete dans http //www.services.eaufrance.fr/observatoire/indicateurs/p103.2a Indice d ’ avancement de la protection de la ressource en eau Cet indicateur traduit l'avancement des démarches administratives et de terrain mises en ceuvre pour protéger les points de captage - Formule de calcul Moyenne pondérée de I|'Indice d ’ avancement de la protection de chaque ressource par le volume produit par la ressource Taux d ’ occurrence des interruptions de service non programmeées Cet indicateur sert € mesurer la continuité du service d'eau potable en suivant le nombre de coupures d ’ eau impromptues pour lesquelles les abonnés concernés n ’ ont pas été prévenus au moins 24h a l'avance , rapporté a 1000 abonnés - Formule de calcul Nombre d'interruptions de service non programmées / Nombre d'abonnés x 1000 Indice linéaire de réparation L'indice linéaire de réparations évalue , en les rapportant a la longueur des canalisations hors branchements , les réparations effectuées sur les réseaux - Formule de calcul Nombre de ruptures sur conduites principales / linéaire de l ’ inventaire des collectivités uniquement canalisations Indice lineaire de pertes Moyenne journaliére des pertes rapportée a |l ’ inventaire total des réseaux du périmétre Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 12

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE SYNTHESE DES ACHATS ET VENTES D ’ EAU Synthese de l'ensemble des volumes qui permet de calculer les indicateurs de rendement réseau , les indices linéaires de pertes et de volumes non comptees presentes dans la parties performance du réseau . OP Pg Mn , ORO ee PPP ae PRELEVE PRODUIT 2 027 575 mé 2 027 575 m EXPORTE 113 354 m IMPORTE 0 m* MIS EN DISTRIBUTION 1 914 221 m* PERTES ® 558 591 m® AUTORISE NON COMPTE 194 000 m CONSOMME FACTURE 1 161 630 m° SYNTHESE ARS 2018 voir ci-aprés Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 13

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE Liste des indicateurs et résultats D101.0 Estimation du nombre d'habitants desservis 19 575 D102.0 Prix TTC du service au m3 pour 120 m3 valeur au 01/01/2018 1,53 € TTC D151.0 Délai maximal d'ouverture des branchements pour les nouveaux abonnés défini par le service 48h aux de conformité des prélevements sur les eaux distribuées réalisés au titre du contrdle sanitaire par rapport aux limites de qualité pour ce qui concerne la microbiologie P1044 OU pour les services desservant moins de 5 000 habitants et produisant moins de 1 000 m3/j 46 Nombre de prélevements aux fins d'analyses microbiologiques effectués dans l'année Nombre de prelevements non conformes parmi Ceux-ci aux de conformité des prélevements sur les eaux distribuées réalisés au titre du contrdéle sanitaire par rapport aux limites de qualité pour ce qui concerne les parametres physico-chimiques 100 % Oo OU pour les services desservant moins de 5 000 habitants et produisant moins de 1 000 m3/j chimiques effectués dans l'année p48 P103.2 Indice de connaissance et de gestion patrimoniale des réseaux d'eau potable 105 P102.1 46 Zz Oo 3 oO Q To om Ds WD 3 SD -- n ” ® Cc x SD n Q ® S o n © n ” To a oO Oo Z O 3 o oe Q Oo en Os ® © 3 S - a D ° S Q O O ~— 5 © n Oo © “ 3 Q Cc x Q. P104.3 Rendement du réseau de distribution 72 % P105.3 |Indice linéaire des volumes non comptés m3/km/j 19,9 m /km/j P106.3 |Indice linéaire de pertes en réseau m3/km/j 14,8 m /km/j Zz O P107.2 |Taux moyen de renouvellement des réseaux d'eau potable P108.3 |Indice d ’ avancement de la protection de la ressource en eau 80 % P109.0 Montant des abandons de créances ou des versements a un fond de solidarité 143 € P151.1 Taux d ’ occurrence des interruptions de service non programmeées 2,84 % o P152.1 |Taux de respect du délai maximal d ’ ouverture des branchements pour les nouveaux abonnés 100 % P153.2 Durée d ’ extinction de la dette de la collectivité 0,3 an P154.0 /Taux d'impayés sur les factures d ’ eau de l'année precedente 2,13 % P155.1 /Taux de réclamations 0,15 % bo Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 14

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE RUPTURES 2018 SELESTAT Rue du Thannenkirch Branchement 19/03/2018 Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 15

EAU POTABLE 2018 311 ee TERRITOIRE ALSACE CENTRALE SELESTAT Au Dieweg Branchement 19/10/2018 Rapport Annuel 2018 sur le Prix et la Qualité du Service d ’ Eau Potable - renvoi vers le glossaire- sdea.fr 16

eee =e a Syndicat des Eaux et de ’ Assainissement Alsace Moselle SDEA Perimetre de SELESTAT ORIGINE DE L ’ EAU La Ville de Sélestat 19181 habitants ’ est alimentée en eau par 1 forage . Cette ressource en eau a été déclarée d'utilité publique le 09 octobre 2000 et dispose de périmétres de protection . Le réseau d ’ eau potable est exploité par le Syndicat des Eaux et de |l ’ Assainissement Alsace Moselle SDEA depuis le 1 “ janvier 2018 . L ’ eau fait objet d ’ un traitement de désinfection au chlore avant sa distribution . Les prélevements d'eau sont réalisés au captage , au réservoir et sur le réseau de distribution . 1 population au 01/01/2011 données INSEE QUALITE DE L ’ EAU DU ROBINET 49 prelevements d'eau ont été réalisés par le Centre d'Analyses et de Recherches , laboratoire agréé par le ministere chargé de la santé . BACTERIOLOGIE Absence exigée de bacteéries indicatrices de pollution r uniquemer 44 analyses bactériologiques réalisées sur l'ensemble du réseau d ’ eau potable . eau d'eau frode 0 analyse non-conforme aux limites de qualité reglementaires . Taux de conformité 100 % Eau de tres bonne qualité microbiologique . DURETE , PH Référence de qualité pH 6,5 a9 Dureté 26,2 °f degré francais '' pH 7,6 Eau dure calcaire et a l ’ equilibre ou legerement agressive et proche de ’ équilibre . NITRATES Limite de qualité 50 mg/I » Teneur moyenne 11,6 mg/| » Teneur maximale 12,1 mg/| Ces valeurs temoignent d'une ressource bien protégée des apports en nitrates . CHLORURES , SODIUM ET FLUOR Références de qualité . Teneur moyenne en chlorures 100,9 mg/| Chlorures 250 mg/l Ss résultats d analyses Teneur moyenne en sodium 25,1 mg/l Sodium 200 mg/I upotable sante gouv fr Teneur moyenne en fluor 0,06 mg/I Fluor 1,5 mg/l PESTICIDES Limite de qualité 0,1 ug/ Les pesticides recherchés n'ont pas ete détectés . ile de Santé Grand Est MICROPOLLUANTS — SOLVANTS — RADIOACTIVITE — AUTRES PARAMETRES Peale du Bas-Rhin Limite s de qualité propre s a chaque parametre Juin Les résultats pour les paramétres mesurés sont conformes aux limites de qualités en vigueur . t67-vsse ars.sante.fr CONCLUSION SANITAIRE 5 79 86 En 2018 , l'eau produite et distribuée par le SDEA - Périmetre de SELESTAT est conforme aux otolia.com limites de qualité bactériologiques et physico-chimiques en vigueur . Cette fiche , destinée aux abonnés du service public de distribution de l ’ eau , peut étre reproduite sans suppression , ni ajout . Dans les immeubles collectifs , elle doit étre distribuée 4 chaque locataire ou affichée .

re d'analyses eflecluées pour le controle saniaire dépend du nombre d ’ habitants desservis et du debit de la ressource forage ou source , Les prélévements sont réalisés 4 la ressource , 4 la production en sortic de station de traitement par exemple et.sur le zu d ¢ distribution réservoir de stockage et robinet du consommateur . ver potenticl pour la sanié des personnes UALITE BACTERIOLOGIQUE elle est évaluée par la tion révéle une contamination survecnue soil au niveau source soil en cours de transport , Une absence de nt , un dysfonctionnement momentané des installations de nt d'eau ou une insulfisance d'entretien des ouvrages ESTICIDES la présence de pesticides dans les ressources d'une mauvaise maitrise des produits utilisés pour f les récoltes ou pour désherber , Par précaution , la valeur aire , res basse , est inférteure au x seuil s de toxicité .RSENIC arsenic est un ¢lément d'origine naturelle , ent répandu dans la crotte terrestre et present a l ’ ¢tat de ans loule matiére vivante , C ’ est un élément classé comme i . IL peut entrainer également des troubles cardio- scl neurologiques , IENTS METALLIQUES il s ’ agit en particulier du cadmium , mercure , chrome , curvre , nickel et fer . Leur “ el toxicologique dépend de leur forme chimique , de leur ion , du contexte environnemental et de la possibilité de dans le corps humuin . PRES RECOMMANDATIONS SANITAIRES rver la qualité de votre eau 5 Gtre corrosive en sortie de '' installation de traitement . mmité de l'eau est établic en comparant la concentration de certains paramétres 4 des limites de qualité ou a des références de qualité limite de qualité est une valeur seuil 4 respecter impérativement portant sur des parameétres microbiclogiques et chimiques constituant éérence de qualité est une valeur seuil salisfaire portant sur des parameires microbiologiques , chimiques el radiologiques , clablies a os fins de suivi des installations de production et de distribution d'eau . —= ee la dureté représente les concentrations en calcium ¢t en magnésium présents naturellement dans l eau de la ressource . Elle est sans incidence sur la santé . Au contraire , le calcium et le magnesium jouecni um role important dans la physiclogie humaine et leur apport par lalimentation est essentiel . Dans le cas d*une eau ayant une dureté de moins de 10°F , Vinstallation d'un dispositif d ’ adoucissement de l'eau ne s¢ justifie pas . Sica le sodium est un metal trés réepandu dans la crodite terrestre . Tl est toujours associé 4 d'autres éléments chimiques et principalement aux chlorures . Cet élément vital participe ades fonctions physiclogiques essentielles , oe les chlorures , irés répandus dans la nature , sont des composes naturels des eaux . 11s sont peu toxiques mais peuvent 4 des doses élevécs nuire au gott de `` eau et favoriser la corrosion des canalisalions . Busice le Muor est un oligo-tlément présent naturellement dans eau . Des doses modérées sont bénéfiques pour la santé , La valeur limie réglementaire a été fixée pour tenir compte du risque de Nuorose dentaire trace sur émail des dents , Lorsque cau est peu fluorée , une prévention ophimale de la cane dentaire passe par un apport complémentaire de cet élément sel Huoré , dentifrice fluaré , comprimeés . ee ORGANIQOUES VOLATILS COV les COV sont des molécules de la chimie de synthese , derivés des hydrocarbures , ou des éléments issus de la dégradation de ces molécules . Les COV peuvent avoir , a long terme , des effets lerulogenes , mulavenes ou Cunceropencs , ¢ les matériaux constitutifs de vos canalisations et les faire changer s'il y a du plomb et ne jamais raccorder l'installation électrique 4 la ie pour faire prise de terre . Ce raccordement peut provoquer des phénomeénes électriques accentuant la corrosion des matériaux . Dans 8 , tl esl recommandeé de contacter un eleciricien professtonnel avant loule intervention . tement complémentaire purificateur , osmoscur cst installé , « doit étre réguligrement entrctenu et réghé par un installateur afin quill n ' y ait pas de risque de dégradation de la qualité microbiologique ou physico-chimique de l'eau lid ce dispositif , L ’ eau ne En savoir plus sur tous les résultats d ’ analyse du contréle sanitaire de l ’ eau sur www.eaupotable.sante.gouv.fr Agence Régionale de Santé Grand Est - 3 , boulevard Joffre - CO 80071- 54 036 NANCY CEDEX
"""
text = prepro(text)

questions = [
#                 "Combien de communes membres dans SDEA ?",
#                 "Depuis quand le président est membre de SDEA ?",
#                 "Quel est le volume total consommé dans SDEA ?",
#                 "Quel est la consommation du volume par habitant ?",
#                 "quel est la capacité de production ?",
#                 "Quel est le taux de mobilisation jour moyen ?",
#                 "quel est le taux de conformité physico-chimique ?",
#                 "quel est le taux d'occurrence des interruptions de service non programmées ?",
#                 "Combien d'appareil de defense contre l'incendie existe ?",
#                 "Quel est le rendement du réseau de distribution ?",
#                 "Quel est le taux de comfirimité des bacteriologie ?",
#                 "Qu'est ce que signifie le taux d’occurrence des interruptions de service non programmées ?",
#                 "Qu'est ce que signifie l'indice d’avancement de la protection de la ressource en eau ?",
# #                 "Quelle est la capacité totale de stockage d'un résevoir ?",
#                 "Quel est le volume utile des résevoirs ?",
#                 "Quel est l'indice d’avancement de la protection de la ressource en eau en 2018 ?",
#                 "quelle est l'autonomie de réservoir ?",
#                 "quelle est le taux de conformité physico-chimique en 2017 ?",
                    "Quel est le taux d’impayés sur factures d’eau de l’année précédente en 2017 ?",
    "Quel est le taux de réclamations global en 2017 ?",
    "Quel est l'indice linéaire des volumes non comptés ?",
    "Combien de compteurs remplacés dans le parc ?",
    "Quels sont travaux de renouvellement du réseau d’eau potable avant réfection lourde de voirie ?",
    "Qu'est ce que signifie AERM ?",
    "Quel est le taux de réclamations ?",
    "Quel est le rendement du réseau de distribution ?"
            ]

print(text)


eau potable 2018311 ee territoire alsace centrale votre commission locale votre president francis franck au cours de l'année 2018, la dynamique d'adhesions et d'intéegrations au sdea s'est poursuivie, traduisant ainsi la confiance croissante exprimée a l'egard de notre outil commun par de nombreuses communes et établissements publics de coopération intercommunale epc. le sdea compte ainsi désormais 668 communes membres au total, fedérant plus de 1000000 d'habitants. ces adhesions, dans les domaines historiques de i'eau potable et'assainissement, mais aussi dans le domaine plus recent du grand cycle de i'eau, permettent a la fois une capacité renforcee de mutualisation au sdea au benéfice de tous nos usagers, et une opportunite de repondre aux enjeux de l'eau dans les différents territoires, en partenariat avec les intercommunalités et les acteurs cles. dans le contexte marque par le changement climatique et les nouveaux défis environnementaux nécessitant une approche encore plus intég

In [59]:
for q in questions:
    print("La question est: "+q)
    print("La résponse est: "+select_answer(question = q,text = text))

Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors


La question est: Quel est le taux d’impayés sur factures d’eau de l’année précédente en 2017 ?


Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors


La résponse est: <s>
La question est: Quel est le taux de réclamations global en 2017 ?


Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors


...

La résponse est: <s>
La question est: Qu'est ce que signifie AERM ?


Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors


La résponse est: perimetre de selestat
La question est: Quel est le taux de réclamations ?


Token indices sequence length is longer than the specified maximum sequence length for this model (5668 > 512). Running this sequence through the model will result in indexing errors


La résponse est: 0, 15% 0
La question est: Quel est le rendement du réseau de distribution ?
La résponse est: 72%
